In [4]:
import numpy as np
import pandas as pd
import math
import string
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD 
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.feature_extraction import DictVectorizer
# from xgboost.sklearn import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [23]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import time
start = time.time()

meta = pd.read_csv("./AutoKaggle - Metadata.csv")
arrOfRows = [64,360,239,317]
nlp_rows = [239]
row = 317
train = ''
def preprocessing(row):
    print(meta['name'].loc[row])
    find_row = meta.loc[row]
    train = ''
    test = None
    check_test = True
    train_X = ''
    train_Y = ''
    test_X = None
    if meta['name'].loc[row] == 'kobe-bryant-shot-selection':
        train = pd.read_csv("./kobe-bryant-shot-selection/data/data.csv")
        check_test = False
    elif meta['name'].loc[row] == 'mercedes-benz-greener-manufacturing':
        train = pd.read_csv("./mercedes-benz-greener-manufacturing/data/train.csv")
        test = pd.read_csv("./mercedes-benz-greener-manufacturing/data/test.csv")
    elif meta['name'].loc[row] == 'uciml_sms-spam-collection-dataset':
        train = pd.read_csv("./uciml_sms-spam-collection-dataset/data/spam.csv", encoding='cp1252', error_bad_lines=False)
        check_test = False
        
    
    if check_test:
        test = test.dropna()
    if meta['name'].loc[row] == 'uciml_sms-spam-collection-dataset':
        row = pd.read_csv("./uciml_sms-spam-collection-dataset/submission/row.csv", encoding='cp1252')
        sms = train
        row_prepro = row['preprocessing function call'][0]
        prepro_ls = eval(row_prepro)
        sms = eval(prepro_ls[0])
        train = eval(prepro_ls[1])
        return train
    else:
        train = train.dropna()
        for c in train.columns:
            if train[c].dtype == 'object':    # deal with text
                lbl = LabelEncoder() 
                if check_test:
                    lbl.fit(list(train[c].values) + list(test[c].values)) 
                    train[c] = lbl.transform(list(train[c].values))
                    test[c] = lbl.transform(list(test[c].values))
                else:
                    lbl.fit(list(train[c].values))
                    train[c] = lbl.transform(list(train[c].values))

        targetName = find_row['targetName']
        train_Y = train[targetName]
        train_X = train.drop(columns=targetName)
        if check_test:
            test_X = test
            return train_X, train_Y, test_X
        else:
            return train_X, train_Y, None


preprocessing(row)


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd5 in position 61: invalid continuation byte

In [24]:
def text_process(text):
    
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    
    return " ".join(text)

def feature_extraction(row, X_train, X_test):
    if meta['name'].loc[row] == 'uciml_sms-spam-collection-dataset':
        rowcsv = pd.read_csv("./uciml_sms-spam-collection-dataset/submission/row.csv")
        row_extract = rowcsv['featureExtractor function call'].loc[0]
        sms = X_train
        extract = eval(row_extract)
        sms['message'] = eval(extract[0])
        sms['message'] = eval(extract[1])
        text_feat = sms['message'].apply(str).copy()
        text_feat = eval(extract[2])
        vectorizer = eval(extract[3])
        features = eval(extract[4])
        features_train, features_test, labels_train, labels_test = train_test_split(features, sms['label'], test_size=0.3)
        return features_train, features_test, labels_train, labels_test
    else:
        if type(meta["featureExtractor function call"].loc[row]) is not str:
            print('not func')
            return X_train,X_test
        extraction_function_calls = str(meta["function call feature extraction"].loc[row])
        extraction_function_calls = extraction_function_calls.split(",")
        extraction_funtion_param = eval(meta["function parameters feature extraction"].loc[row])
        function_nums = len(extraction_function_calls)
        for i in range(function_nums):
            str1 = extraction_function_calls[i]
            str2 = extraction_funtion_param[i]
            l_str = str1.split("(")
            l_str.insert(1,"("+str2)
            str_call = ''
            str_call = str_call.join(l_str)
            str_call = 'extractor' + '=' + str_call
            exec(str_call, globals(), globals())
            extracted_train = extractor.fit_transform(X_train)
            n_comp = extracted_train.shape[1]
            for j in range(0, n_comp):
                X_train['extractor'+ str(i)+"_"+str(j)] = extracted_train[:, j]
            if X_test is not None:
                extracted_test = extractor.fit_transform(X_test)
                for j in range(0, n_comp):
                    X_test['extractor'+ str(i)+"_"+str(j)] = extracted_test[:, j]
                return X_train,X_test
            else:
                return X_train,None

In [16]:
train = preprocessing(row)
feature_extraction(row, train, None)

NameError: name 'row' is not defined

In [17]:
def feature_selection():
    pass

In [18]:
def estimation(row,X_train,X_test,Y_train, Y_test):
    if row in nlp_rows:
        rowcsv = pd.read_csv("./uciml_sms-spam-collection-dataset/submission/row.csv")
        row_extract = eval(rowcsv['estimator1 function call'].loc[0])
        mnb = eval(row_extract[0])
        eval(row_extract[1])
        pred = eval(row_extract[2])
        if rowcsv['performanceMetric'].loc[0] == 'accuracy':
            return accuracy_score(Y_test, pred)
    else:
        estimation_function_calls = eval(meta["estimator1 function call"].loc[row])
        print(len(estimation_function_calls))
        if len(estimation_function_calls) == 1:
            str_call = estimation_function_calls[0]
            print(str_call)
            str_call = 'estimator' + '=' + str_call
            print(str_call)
            exec(str_call,globals(),globals())
            
            if meta["taskType"].loc[row] == 'classification':
                estimator.fit(X_train,Y_train)
                Y_pred = estimator.predict(X_test)
                print('here')
                print(recall_score(Y_test,Y_pred,average='weighted'))
                print('here')
            elif meta["taskType"].loc[row] == 'regression':
                estimator.fit(X_train,Y_train)
                print(r2_score(Y_test,Y_pred,average='weighted'))
        else:
            estimators = []
            n_estimators = len(estimation_function_calls)
            for i in range(n_estimators):
                str1 = extraction_function_calls
                str2 = extraction_funtion_param
                l_str = str1.split("(")
                l_str.insert(1,"("+str2)
                str_call = ''
                str_call = str_call.join(l_str)
                str_call = 'estimator' + '=' + str_call
                print(l_str)
                print(str_call)
                exec(str_call)
                estimators.append(estimator)
                postprocessing(estimators,stack = True)
#     if row in nlp_rows:
#         rowcsv = pd.read_csv("./uciml_sms-spam-collection-dataset/submission/row.csv")
#         row_extract = eval(rowcsv['estimator1 function call'].loc[0])
#         mnb = eval(row_extract[0])
#         eval(row_extract[1])
#         pred = eval(row_extract[2])
#         if rowcsv['performanceMetric'].loc[0] == 'accuracy':
#             return accuracy_score(Y_test, pred)
#     else:
#         estimation_function_calls = meta["function calls estimation"].loc[row]
#         estimation_function_calls = estimation_function_calls.split(",")
#         print(type(meta["function parameters estimation"].loc[row]))
#         print(meta["function parameters estimation"].loc[row])
#         estimation_function_param = eval(meta["function parameters estimation"].loc[row])

#         print(len(estimation_function_calls))
#         if len(estimation_function_calls) == 1:
#             l_str = estimation_function_calls[0].split("(")
#             l_str.insert(1,'('+estimation_function_param[0])
#             str_call = ''
#             str_call = str_call.join(l_str)
#             str_call = 'estimator' + '=' + str_call
#             exec(str_call,globals(),globals())
#     #         estimator.fit(X_train,Y_train)
#             print(cross_val_score(estimator, X_train, Y_train, cv=3, n_jobs=8))
#         else:
#             estimators = []
#             n_estimators = len(estimation_function_calls)
#             for i in range(n_estimators):
#                 str1 = extraction_function_calls
#                 str2 = extraction_funtion_param
#                 l_str = str1.split("(")
#                 l_str.insert(1,"("+str2)
#                 str_call = ''
#                 str_call = str_call.join(l_str)
#                 str_call = 'estimator' + '=' + str_call
#                 print(l_str)
#                 print(str_call)
#                 exec(str_call)
#                 estimators.append(estimator)
#                 postprocessing(estimators,stack = True)

In [19]:
train_set = preprocessing(row)
X_train, X_test, Y_train, Y_test = feature_extraction(row, train_set, None)
Y_pred = estimation(row, ['shot_made_flag'], None, None, None)
print(Y_pred)

NameError: name 'row' is not defined

In [20]:
if meta['name'].loc[row] == 'uciml_sms-spam-collection-dataset':
    train_set = preprocessing(row)
    X_train, X_test, Y_train, Y_test = feature_extraction(row, train_set, None)
    Y_pred = estimation(row, X_train, X_test, Y_train, Y_test)
    print(Y_pred)

NameError: name 'meta' is not defined

In [21]:
def postprocessing(estimators,stack):
    pass

In [22]:
end = time.time()
print("Use", end - start, "seconds to run this.")

Use 2.0979933738708496 seconds to run this.
